# Phages attack for sure the bacterium who entered the system last before them and every other bacterium with some probability.
Beta is a characteristic of the phage

In [ ]:
import numpy as np
#import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import linalg
import seaborn as sns
import random 
from random import randint
%matplotlib inline
#from scipy.integrate import solve_ivp
import timeit
import time
#import cProfile
#%load_ext line_profiler
#from numba import jit
#from numba.typed import List
%load_ext cython
#import pixiedust

In [ ]:
#pixiedust.optOut()

In [ ]:
%%cython -a


import numpy as np
cimport numpy as np
import cython


########################################   1st function

@cython.cdivision(True) # We promise that we will not devide by zero
@cython.boundscheck(False) # Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False) #  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.


def rk4(double[:] pop, double t, double Dt, long long[:] species, double[:] dS_dt, long n,
        double[:] k, double[:] Beta, double alpha, double Delta, double[:] temp, double[:] result, 
        np.ndarray[double, ndim=2] eta_matrix):
    ''' Runge-Kutta 4th order method'''
    '''We implement it to see which species goes extinct and what will the populations be afterwards'''
    '''Cython implementation which additionally changes the population and the rate of change of the population'''
    # We make sure that every variable or constant that will be used has its type defined and that everything is local.
    
    cdef int num_p = 0
    cdef int num_b = 0
    
    for i in range(n):
        num_p += species[i] 
    
    num_b = n - num_p
    
    t = 0.0
    
    while True:

        
        # Implementing the Runge-Kutta by calling the growth rate function
        for i in range(n):
            result[i] = 0

        g_r(t, pop, species, dS_dt, n, num_b, num_p, k, Beta, alpha, Delta, eta_matrix)
        for i in range(n):
            result[i] += dS_dt[i] / 6

        for i in range(n):
            temp[i] = pop[i] + Dt*dS_dt[i]/2
        g_r(t + Dt / 2, temp, species, dS_dt, n, num_b, num_p, k, Beta, alpha, Delta, eta_matrix)
        for i in range(n):
            result[i] += dS_dt[i] / 3

        for i in range(n):
            temp[i] = pop[i] + Dt*dS_dt[i]/2
        g_r(t + Dt / 2, temp, species, dS_dt, n, num_b, num_p, k, Beta, alpha, Delta, eta_matrix)
        for i in range(n):
            result[i] += dS_dt[i] / 3

        for i in range(n):
            temp[i] = pop[i] + Dt*dS_dt[i]
        g_r(t + Dt, temp, species, dS_dt, n, num_b, num_p, k, Beta, alpha, Delta, eta_matrix)
        for i in range(n):
            result[i] += dS_dt[i] / 6

        for i in range(n):
            pop[i] += result[i] * Dt

        # Check whether any species has gone extinct
        for i in range(n):
            if pop[i] < 1e-20:
                return

        t += Dt
        
        

########################################   2nd function
        
@cython.cdivision(True) # We promise that we will not devide by zero
@cython.boundscheck(False) # Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False) #  and that we won't be using negative indexes (the Python syntax for indexing from the end).
#These will prevent a bunch of error checking code from being added.

cdef void g_r(double t, double[:] pop, long long[:] species, double[:] dS_dt, long n, long num_b, long num_p,
              double[:] k, double[:] Beta, double alpha, double Delta, np.ndarray[double, ndim=2] eta_matrix):
    '''Growth rate calculation'''
    '''Cython implementation of the growth rate function. Returns nothing, but it changes the growth rate at each step dS_dt'''

    cdef double sumB1, sumB2, sumP, popj
    cdef int i, j, SumPhage, SumBact
    cdef int Num_B = 0
    cdef int Num_P = 0

    for i in range(n):
        
        if species[i] == 0:
            dS_dt[i] = k[i]*pop[i] - alpha*pop[i]
            sumB1 = k[i]*pop[i]*pop[i]
            sumB2 = 0
    
            for j in range(n):
                if j != i:
                    if species[j] == 0:
                        sumB1 += k[i]*pop[i]*pop[j]
            for j in range(num_p):
                SumPhage = -1
                for o in range(n):
                    if species[o] == 1:
                        SumPhage += 1
                        if SumPhage == j:
                            popj = pop[o]
                            break
                            
                sumB2 += pop[i]*eta_matrix[j,Num_B]*popj
                    
            Num_B += 1            
            dS_dt[i] -= sumB1 + sumB2
            
        elif species[i] == 1:
            
            dS_dt[i] = -Delta*pop[i]
            sumP = 0
            
            for j in range(num_b):
                SumBact = -1
                for o in range(n):
                    if species[o] == 0:
                        SumBact += 1
                        if SumBact == j:
                            popj = pop[o]
                            break
                    
                sumP += pop[i]*Beta[i]*eta_matrix[Num_P,j]*popj
                    
    
            Num_P += 1
            dS_dt[i] += sumP

        

In [ ]:
%%cython -a

import numpy
cimport numpy as np
import cython
from libc.math cimport fabs
import random

@cython.cdivision(True)# We promise that we will not devide by zero
@cython.boundscheck(False)# Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False)#  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.

def check4eq(long n, double[:] k, double[:] Beta):
    "Check that every newly introduced species doesn't already exist in the system."
    "Introduce another species in its place if that is the case"
    
    cdef int CHECK = 2
    
    while CHECK == 2:
        CHECK = 1
        if Beta[n-1] != 0:
            for i in range(n-1):
                if Beta[n-1] == Beta[i]:
                    #print('CHANGE PHAGE')
                    Beta[n-1] = random.randint(1, 50)
                    CHECK = 2

        else:
            for i in range(n-1):
                if (fabs(k[i] - k[n-1])/k[i]) < 0.001:
                    #print('CHANGE BACTERIUM')
                    k[n-1] = random.uniform(0, 1)
                    CHECK = 2
                
    
    

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np
import cython
import random

@cython.cdivision(True)# We promise that we will not devide by zero
@cython.boundscheck(False)# Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False)#  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.

def Eta_Matrix(np.ndarray[double, ndim=2] eta_matrix, long n, long long[:] species):
    "Calculate the eta matrix"
    cdef int num_p = 0
    cdef int num_b = 0
    cdef int i,j
    cdef float SUM, Prob
    
    
    for i in range(n):
        num_p += species[i] 
    
    num_b = n - num_p
    
    if species[n-1] == 0:
        if num_p > 0 and num_b > 0:
            eta_matrix = np.c_[eta_matrix,np.zeros(num_p)]
        else:
            eta_matrix = np.zeros([num_p, num_b],dtype=np.double)
    else:
        if num_p > 0 and num_b > 0:
            eta_matrix = np.r_[eta_matrix,[np.zeros(num_b)]]
            for j in range(num_b):
                SUM = 0
                for i in range(num_p-1):
                    SUM += eta_matrix[i,j]
                if SUM == 0:
                    eta_matrix[num_p-1, j] = random.uniform(0,1)
            for i in range(num_b):
                Prob = random.uniform(0,1)
                if Prob < 0.4:
                    eta_matrix[num_p-1,i] = random.uniform(0,1)
        else:
            eta_matrix = np.zeros([num_p, num_b],dtype=np.double)
            
         
    return eta_matrix                

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np
import cython


@cython.cdivision(True) # We promise that we will not devide by zero
@cython.boundscheck(False) # Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False) #  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.


def n1(double[:] pop, long long[:] species, long n, double[:] k, double alpha, long equillibrium):
    '''Checking if equillibrium is possible for the characteristics of the first species'''
    #equillibrium is only possible if the first species in the system is B with specific characteristics
    
    if species[n-1] == 0: 
        if (k[n-1] - alpha)/k[n-1] > 0:
            pop[n-1] = (k[n-1] - alpha)/k[n-1]
            equillibrium = 1
            return equillibrium
        else:
            pop[n-1] = 0
            equillibrium = 2
            return equillibrium
    elif species[n-1] == 1:
        pop[n-1] = 0
        equillibrium = 2
        return equillibrium
            

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np
import cython


@cython.cdivision(True) # We promise that we will not devide by zero
@cython.boundscheck(False) # Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False) #  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.

def species_and_K(long long[:] species, long n, double[:] k, double[:] Beta, double[:] K):
    '''Calculate number of bacteria s, number of phages sp and array K'''
    cdef int s = 0
    cdef int sp = 0
    
    for i in range(n):
        if species[i] == 0:
            K[s] = k[i]
            s += 1

    for i in range(n):
        if species[i] == 1:
            K[s+sp] = Beta[i]
            sp += 1

    return sp, s

In [ ]:
%%cython -a


import numpy as np
cimport numpy as np
import cython


@cython.cdivision(True) # We promise that we will not devide by zero
@cython.boundscheck(False) # Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False) #  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.


def matrixA(long long[:] species, long n, np.ndarray[double, ndim=2] A, double[:] K, 
            np.ndarray[double, ndim=2] eta_matrix, long s):
    '''Calculate the interaction matrix A by creating the matrix eta, which keeps track of the interaction
       between each phage and each bacteria'''
    cdef int i, j
    
    for i in range(n):
        for j in range(n):
            if i < s and j < s:
                A[i,j] = 1
            elif i >= s and j >= s:
                A[i,j] = 0
            elif i < s and j >= s:
                A[i,j] = eta_matrix[j-s,i]/K[i]
            else:
                A[i,j] = K[i]*eta_matrix[i-s,j]

    #returning matrix A          
    return A


In [ ]:
%%cython -a


import numpy as np
cimport numpy as np
import cython


@cython.cdivision(True) # We promise that we will not devide by zero
@cython.boundscheck(False) # Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False) #  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.

def Eq_Or_Dead(double[:] pop, long long[:] species, double[:] K, double[:] S, long n, np.ndarray[double, ndim=2] InvA, 
               double alpha, double Delta, long equillibrium):
    '''We calculate whether coexisting steady states are possible for the species of the current system
       based on the matrix product of the inverse of the interaction matrix and the K matrix (see paper).
       If not then at least one species will get extinct'''

    cdef int s, sp, sp2
    cdef double keepit
    
    equillibrium = 2
    #Finding the number of bacterial species
    s = 0
    for i in range(n):
        if species[i] == 0:
            s += 1
    
    #Constructing K matrix
    for i in range(n):
        if i < s:
            keepit = (K[i]-alpha)/K[i]
            K[i] = keepit
        else:
            K[i] = Delta

    #Matrix product
    sp = 0
    for i in range(n):
        for j in range(n):
            S[i] += InvA[i,j] * K[j]

        if S[i] > 0:
            sp += 1
        
    #Every species' population needs to be positive in order to have reached equillibrium
    if sp == n:
        sp = 0
        sp2 = 0
        for i in range(n):
            if species[i] == 0:
                pop[i] = S[sp]
                sp += 1
            else:    
                pop[i] = S[s+sp2]
                sp2 += 1

        equillibrium = 1
    
    return equillibrium


In [ ]:
%%cython -a

import numpy as np
cimport numpy as np
import cython

@cython.cdivision(True) # We promise that we will not devide by zero
@cython.boundscheck(False) # Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False) #  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.

def Resize_Eta(double[:] pop, long long[:] species, long n, np.ndarray[double, ndim=2] eta_matrix):
    '''Resizing eta matrix'''
    cdef int i = 0
    cdef int num_b = 0
    cdef int num_p = 0
    
    for i in range(n):
        if species[i] == 0:
            if pop[i] < 1e-20:
                eta_matrix = np.delete(eta_matrix, num_b, axis = 1)
                num_b -= 1
                
            num_b += 1
        else:
            if pop[i] < 1e-20:
                eta_matrix = np.delete(eta_matrix, num_p, axis = 0)
                num_p -= 1
                
            num_p += 1
        
        
    return eta_matrix    

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np
import cython
from libc.math cimport fabs

@cython.cdivision(True) # We promise that we will not devide by zero
@cython.boundscheck(False) # Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False) #  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.

def sp_dist(long n, long death_note, double[:] per_cent_extinct, long long[:] N, long prev_n, long long[:] n_diff, 
            long long[:] Death_num, long n_max, long long[:] species, long break_nar):
    '''Calculating basic numbers for the distribution of the species and
   the probability to break the narrowing staircase of coexistence'''

    cdef int o

    if n > n_max:
        n_max = n

    if n > 0:
        N[n-1] += 1
        
    if death_note > 0:
        Death_num[death_note-1] += 1
        per_cent_extinct = np.append(per_cent_extinct, death_note / (float(n + death_note)))
        death_note = 0    
        
    if fabs(n-prev_n) != 0.:
        if (n - prev_n) > 0:
            n_diff[(n - prev_n) - 1] += 1
        else:
            n_diff[(prev_n - n) - 1] += 1
    for o in range(1, n):
        if species[o-1] == species[o]:
            break_nar += 1
            break

    return per_cent_extinct, death_note, n, Death_num, N, n_diff, n_max, break_nar        

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np
import cython

@cython.cdivision(True) # We promise that we will not devide by zero
@cython.boundscheck(False) # Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False) #  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.

def keep_track(long n, long c, double[:] k, double[:] Beta, long long[:] species, double[:] pop, 
               np.ndarray[double, ndim=2] Populo):
    '''Keeping track of the species populations and their charasteristics (species, k, beta)'''
    cdef int i
    
    for i in range(n):
        Populo[i,c] = pop[i]
        Populo[i, 10000] = species[i] + 1
        Populo[i,c+10002] = i + 1
        if species[i] == 0:
            Populo[i, 10001] = k[i]
        else:
            Populo[i, 10001] = Beta[i]
        
    return Populo    

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np
import cython

@cython.cdivision(True) # We promise that we will not devide by zero
@cython.boundscheck(False) # Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False) #  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.

def Resize_pop_track(long n, long c, long c2, double[:] pop, np.ndarray[double, ndim=2] Populo, long long[:] position):
    '''Keeping track of the species population and characteristics (species, k, beta)'''
    cdef int i
    cdef np.ndarray inter 
    
    for i in range(n):
        Populo[i,c] = pop[i]
        
        
    if len(position) == 1:
        inter = Populo[position][0]
        inter[c] = 0
        Populo[19999-c2,:] = inter
        Populo[position][0] = 0
        for i in range(position[0], n):
            inter = Populo[i,:]
            Populo[i,:] = Populo[i+1,:]
            Populo[i+1,:] = inter

    elif len(position) == 2:
        inter = Populo[position][0]
        inter[c] = 0
        Populo[19999-c2,:] = inter
        Populo[position][0] = 0
        for i in range(position[0], n):
            inter = Populo[i,:]
            Populo[i,:] = Populo[i+1,:]
            Populo[i+1,:] = inter

        position[1] = position[1] - 1
        inter = Populo[position][1]
        inter[c] = 0
        Populo[12000-c2,:] = inter
        Populo[position][1] = 0
        for i in range(position[1], n):
            inter = Populo[i,:]
            Populo[i,:] = Populo[i+1,:]
            Populo[i+1,:] = inter    
    

    
    return Populo    

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np
import cython

@cython.cdivision(True) # We promise that we will not devide by zero
@cython.boundscheck(False) # Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False) #  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.

def kappa_beta(np.ndarray[double, ndim=2] k_t, np.ndarray[double, ndim=2] Beta_t,
               np.ndarray[double, ndim=2] bacteria_history, np.ndarray[double, ndim=2] phages_history,
               double[:] k_history, double[:] Beta_history, long num_bact, long num_phages):
    
    '''Creating a log of the existent k and Beta at every time step'''
    cdef int i
    cdef int j
    
    for i in range(num_bact):
        for j in range(10000):
            if bacteria_history[i,j] != 0:
                k_t[i,j] = k_history[i]
                
    
    for i in range(num_phages):
        for j in range(10000):
            if phages_history[i,j] != 0:
                Beta_t[i,j] = Beta_history[i] 
                
     
    return k_t, Beta_t

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np
import cython

@cython.cdivision(True) # We promise that we will not devide by zero
@cython.boundscheck(False) # Tells Cython not to put in code to check if the indexes are out of the bounds of the array,
@cython.wraparound(False) #  and that we won't be using negative indexes (the Python syntax for indexing from the end). 
#This will prevent a bunch of error checking code from being added.

def calc_max_k(np.ndarray[double, ndim=2] k_t, double[:] max_k, long num_bact, np.ndarray[long long, ndim=2] bbd1,
               np.ndarray[long long, ndim=2] bbd2):
    '''Calculate the maximum k at every time step as well as the position that the fastest growing bacteria of the
    system changes'''
    cdef int i, j, count1, count2
    cdef float MAX
    
    for j in range(10000):
        MAX = 0
        for i in range(num_bact):
            if k_t[i,j] > MAX:
                MAX = k_t[i,j]
        
        max_k[j] = MAX
        
    MAX = max_k[0]
    count1 = 0
    count2 = 0
    for i in range(1,10000):
        if max_k[i] > MAX:
            bbd1[count1,0] = i
            bbd1[count1,1] = i-1
            MAX = max_k[i]
            count1 += 1
        elif max_k[i] < MAX:
            bbd2[count2,0] = i
            bbd2[count2,1] = i-1
            MAX = max_k[i]
            count2 += 1
    

In [ ]:
def add_new_species(Beta, dS_dt, k, pop, result, species, temp, eta_matrix, n):
    # function for adding new species to the system
    n += 1
    Rand = random.randint(0, 1)
    if Rand == 0:
        species = np.append(species, 0)  # B == 0
        pop = np.append(pop, 1e-6)
        k = np.append(k, random.uniform(0, 1))
        Beta = np.append(Beta, 0)
        dS_dt = np.append(dS_dt, 0)
        temp = np.append(temp, 0)
        result = np.append(result, 0)
        eta_matrix = Eta_Matrix(eta_matrix, n, species)
        
    else:
        species = np.append(species, 1)  # P == 1
        pop = np.append(pop, 1e-6)
        k = np.append(k, 0)
        Beta = np.append(Beta, random.randint(1, 50))
        dS_dt = np.append(dS_dt, 0)
        temp = np.append(temp, 0)
        result = np.append(result, 0)
        eta_matrix = Eta_Matrix(eta_matrix, n, species)
                
    return Beta, dS_dt, k, pop, result, species, temp, eta_matrix, n

In [ ]:
def resize_arrays(Beta, dS_dt, k, pop, result, species, temp, eta_matrix, n, Populo, c, c2):
    # function for taking extinct species out of the system
    mask = pop >= 1e-20
    
    eta_matrix = Resize_Eta(pop, species, n, eta_matrix)
    position = np.where(pop < 1e-20)[0]
    Populo = Resize_pop_track(n, c, c2, pop, Populo, position)
    species = species[mask]
    k = k[mask]
    Beta = Beta[mask]
    dS_dt = dS_dt[mask]
    temp = temp[mask]
    result = result[mask]
    pop = pop[mask]

    return Beta, dS_dt, k, pop, result, species, temp, eta_matrix, Populo

In [ ]:
def major_interactions(k, Beta, eta_matrix, COUNT2_or_more, P_diff_B, B_diff_P, both, same):
    'Checking whether each phage attacks predominantly a different bacteria than the others'
    'Whether each phage has its own main food source'

    Beta_clean = Beta[Beta != 0] #growth rates without zero values
    k_clean = k[k != 0]

    megethosP = len(Beta_clean)
    megethosB = len(k_clean)
    if megethosP > 1:
        COUNT2_or_more += 1
        beta_eta_matrix = np.zeros_like(eta_matrix)
        max_pos_P = np.empty_like(Beta_clean) #position of B where attack of each P is max
        max_pos_B = np.empty_like(k_clean)
        for j in range(megethosP):
            beta_eta_matrix[j,:] = Beta_clean[j] * eta_matrix[j,:]
            max_pos_P[j] = np.where(beta_eta_matrix[j,:] == np.amax(beta_eta_matrix[j,:]))[0][0]
        for j in range(megethosB):
            if np.any(beta_eta_matrix[:,j] != 0):
                max_pos_B[j] = np.where(beta_eta_matrix[:,j] == np.amax(beta_eta_matrix[:,j]))[0][0]
            else:
                max_pos_B = np.delete(max_pos_B, j)

        
        if len(max_pos_P) == len(np.unique(max_pos_P)):
            P_diff_B += 1

        if len(max_pos_B) == len(np.unique(max_pos_B)):
            B_diff_P += 1
        #else:
            #print('HERE =',beta_eta_matrix)

        if len(max_pos_P) == len(np.unique(max_pos_P)) and len(max_pos_B) == len(np.unique(max_pos_B)):
            both += 1
            if np.all(max_pos_P == max_pos_B):
                same += 1
            #else:
                #print(Beta_clean)
                #print(k_clean)
                #print(max_pos_P)
                #print(max_pos_B)
                #print(eta_matrix,'\n')
    
    
    return COUNT2_or_more, P_diff_B, B_diff_P, both, same

In [ ]:
def plots(species, pop, k, Beta, dS_dt, Populo, bacteria_history, phages_history, k_history, Beta_history, 
          num_Pop, num_bact, num_phages, k_t, Beta_t, eta_history, max_k, bbd1, bbd2, per_cent_extinct, N,
          n_diff, Death_num, n_max, break_nar, probability):
    
    #Plot the resulting ecosystem

    #print('Species =',species)
    #print('Population =',pop)
    #print('k =',k)
    #print('Beta =',Beta)
    #print('dS_dt =',dS_dt)
    #print('Populo =', Populo)
    print('number of species that ever existed =', num_Pop)
    #print('Bacteria =', bacteria_history)
    print('number of bacteria that ever existed =', num_bact)
    #print('Phages =', phages_history)
    print('number of phages that ever existed =', num_phages)
    print('Number of times that a fastest growing bacteria was substituted by an even faster growing one =', len(bbd1))
    print('Number of times that a fastest growing bacteria was substituted by an slower growing one =', len(bbd2))
    
    T = np.arange(10000)
    
#     Random = randint(0, len(bbd1))
#     print('eta matrix after the substitution =\n', eta_history[bbd1[Random,0]])
#     print('eta matrix before the substitution =\n', eta_history[bbd1[Random,1]],'\n\n')
#     print('bacterial growth rates at the time = \n', k_t[bbd1[Random,0]])
#     print('bacterial growth rates before the event = \n', k_t[bbd2[Random,1]],'\n\n')
        
    
    
    species_life = np.zeros(num_Pop)
    for i in range(num_Pop):
        species_life[i] = np.count_nonzero(Populo[i,:])
    
    bacteria_life = np.zeros(num_bact)
    for i in range(num_bact):
        bacteria_life[i] = np.count_nonzero(bacteria_history[i,:])
    
    phages_life = np.zeros(num_phages)
    for i in range(num_phages):
        phages_life[i] = np.count_nonzero(phages_history[i,:])    
    print('n_max =',n_max)
    print('p to break the narrowing staircase of coexistence =', break_nar/10000)
    #print('t =',t)
#######################################################begin
    # Plot the dynamics

#     fig1, ax1 = plt.subplots(figsize = [8,6])
#     ax1.set_title('Staircase of coexistence');
#     ax1.set_xlabel('Bacterial strains');
#     ax1.set_ylabel('Phage strains');
#     ax1.grid();
#     ax1.set_xlim(0,6)
#     ax1.set_ylim(-0.2,6)
#     Bact = 0
#     Phage = 0
#     for i in species:
#         if i == 0:
#             Bact += 1
#         else:
#             Phage += 1

#         ax1.plot(Bact, Phage, 'ko', markersize=14)

########################################################end

    # Plot the histogram of the number of species
    fig2, ax2 = plt.subplots(figsize = [8,6]);
    ax2.set_title('Distribution of the number of species', fontsize='x-large',fontweight='bold');
    ax2.set_xlabel('Number of species in the system', fontsize='x-large');
    ax2.set_ylabel('Frequency', fontsize='x-large');
    ax2.set_xlim([0,22]);
    ax2.set_ylim([0,4000]);
    sp_num = np.arange(1,len(N)+1)
    ax2.bar(sp_num, N, align='center', alpha=0.5, color='blue');
    ax2.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    ax2.set_xticks(np.arange(0,24,2));
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig2.savefig("Distribution of number of species (p = "+probability+") (10000 steps).png");
    
####################################################begin  
#     # Plot the evolution of population of species that formed stable systems
#     fig4, ax4 = plt.subplots(figsize = [8,6]);

#     Populo[Populo == 0] = np.nan
#     ax4.grid();
#     ax4.set_title('Evolution of population of species that were part of a stable system at some time');
#     ax4.set_xlabel('time(generations of stable systems)');
#     ax4.set_ylabel('population');
#     ax4.set_yscale('log');
#     for i in range(num_Pop):
#         ax4.plot(T, Populo[i,:],'-ok', markersize=1.5, linewidth=0.65);
        
#     fig4.savefig("Evolution of population of species (p = 0.7).png");  
    
#     # Plot the evolution of population of bacteria that formed stable systems
#     fig6, ax6 = plt.subplots(figsize = [8,6]);

#     bacteria_history[bacteria_history == 0] = np.nan
#     ax6.grid();
#     ax6.set_title('Evolution of population of bacteria that were part of a stable system at some time');
#     ax6.set_xlabel('time(generations of stable systems)');
#     ax6.set_ylabel('population');
#     ax6.set_yscale('log');
#     for i in range(num_bact):
#         ax6.plot(T, bacteria_history[i,:],'-ok', markersize=1.5, linewidth=0.65);
        
#     fig6.savefig("Evolution of population of bacteria (p = 0.7).png");  
    
#     # Plot the evolution of population of phages that formed stable systems
#     fig7, ax7 = plt.subplots(figsize = [8,6]);

#     phages_history[phages_history == 0] = np.nan
#     ax7.grid();
#     ax7.set_title('Evolution of population of phages that were part of a stable system at some time');
#     ax7.set_xlabel('time(generations of stable systems)');
#     ax7.set_ylabel('population');
#     ax7.set_yscale('log');
#     for i in range(num_phages):
#         ax7.plot(T, phages_history[i,:],'-ok', markersize=1.5, linewidth=0.65);
        
#     fig7.savefig("Evolution of population of phages (p = 0.7).png");  
##############################################end 

    # Histogram of the duration of the lives of species
    fig5, ax5 = plt.subplots(figsize = [8,6]);
    ax5.set_title('Distribution of the duration of the lives of species', fontsize='x-large',fontweight='bold');
    ax5.set_xlabel('Life duration', fontsize='x-large');
    ax5.set_ylabel('Frequency', fontsize='x-large');
    ax5.set_xscale('log');
    ax5.set_yscale('log');
    ax5.hist(species_life, color='blue', alpha=0.7, bins=np.logspace(np.log10(1),np.log10(max(species_life) + 1), 10));
    ax5.set_ylim([0,5*10**3]);
    ax5.set_xlim([0,2*10**4]);
    ax5.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig5.tight_layout();
    fig5.savefig("Lives of species (p = "+probability+") (10000 steps).png");

    # Histogram of the duration of the lives of bacteria
    fig8, ax8 = plt.subplots(figsize = [8,6]);
    ax8.set_title('Distribution of the duration of the lives of bacteria', fontsize='x-large',fontweight='bold');
    ax8.set_xlabel('Life duration', fontsize='x-large');
    ax8.set_ylabel('Frequency', fontsize='x-large');
    ax8.set_xscale('log');
    ax8.set_yscale('log');
    max_b_life = max(bacteria_life)
    ax8.hist(bacteria_life, color='blue', alpha=0.7, bins=np.logspace(np.log10(1),np.log10(max_b_life + 1), 10));
    ax8.set_ylim([0,5*10**3]);
    ax8.set_xlim([0,2*10**4]);
    ax8.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig8.tight_layout();
    fig8.savefig("Lives of bacteria (p = "+probability+") (10000 steps).png");
    
    # Histogram of the duration of the lives of bacteria (fast growers vs slow growers)
    fig12, ax12 = plt.subplots(figsize = [8,6]);
    ax12.set_title('Distribution of the duration of the lives of bacteria (fast growers vs slow growers)', fontsize='large',fontweight='bold');
    ax12.set_xlabel('Life duration', fontsize='x-large');
    ax12.set_ylabel('Frequency', fontsize='x-large');
    ax12.set_xscale('log');
    ax12.set_yscale('log');
    meso_b = round(num_bact/2)
    slow_growing_b = bacteria_life[:meso_b]
    fast_growing_b = bacteria_life[meso_b:]
    ax12.hist(slow_growing_b, bins=np.logspace(np.log10(1),np.log10(max_b_life + 1), 10), alpha=0.8, color='green', label='slow growers');
    ax12.hist(fast_growing_b, bins=np.logspace(np.log10(1),np.log10(max_b_life + 1), 10), alpha=0.8, color='red', label='fast growers');
    ax12.set_ylim([0,5*10**3]);
    ax12.set_xlim([0,2*10**4]);
    ax12.legend(title='p = '+probability, fontsize = 'large', title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig12.tight_layout();
    fig12.savefig("Lives of bacteria (fast vs slow) (p = "+probability+") (10000 steps).png");
    
    # Histogram of the duration of the lives of phages
    fig9, ax9 = plt.subplots(figsize = [8,6]);
    ax9.set_title('Distribution of the duration of the lives of phages', fontsize='x-large',fontweight='bold');
    ax9.set_xlabel('Life duration', fontsize='x-large');
    ax9.set_ylabel('Frequency', fontsize='x-large');
    ax9.set_xscale('log');
    ax9.set_yscale('log');
    max_p_life = max(phages_life)
    ax9.hist(phages_life, color='blue', alpha=0.7, bins=np.logspace(np.log10(1),np.log10(max_p_life + 1), 10));
    ax9.set_ylim([0,5*10**3]);
    ax9.set_xlim([0,2*10**4]);
    ax9.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig9.tight_layout();
    fig9.savefig("Lives of phages (p = "+probability+") (10000 steps).png");

    # Histogram of the duration of the lives of phages (fast growers vs slow growers)
    fig13, ax13 = plt.subplots(figsize = [8,6]);
    ax13.set_title('Distribution of the duration of the lives of phages (fast growers vs slow growers)', fontsize='large',fontweight='bold');
    ax13.set_xlabel('Life duration', fontsize='x-large');
    ax13.set_ylabel('Frequency', fontsize='x-large');
    ax13.set_xscale('log'); 
    ax13.set_yscale('log');
    meso_p = round(num_phages/2)
    slow_growing_p = phages_life[:meso_p]
    fast_growing_p = phages_life[meso_p:]
    ax13.hist(slow_growing_p, bins=np.logspace(np.log10(1),np.log10(max_p_life + 1), 10), alpha=0.8, color='green', label='slow growers');
    ax13.hist(fast_growing_p, bins=np.logspace(np.log10(1),np.log10(max_p_life + 1), 10), alpha=0.8, color='red', label='fast growers');
    ax13.set_ylim([0,5*10**3]);
    ax13.set_xlim([0,2*10**4]);
    ax13.legend(title='p = '+probability, fontsize = 'large', title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig13.tight_layout();
    fig13.savefig("Lives of phages (fast vs slow) (p = "+probability+") (10000 steps).png");
    
    # Histogram of k
    fig10, ax10 = plt.subplots(figsize = [8,6]);
    ax10.set_title('Distribution of k', fontsize='x-large',fontweight='bold');
    ax10.set_xlabel('k', fontsize='x-large');
    ax10.set_ylabel('Frequency', fontsize='x-large');
    ax10.hist(k_history, bins=list(np.linspace(0,1,11)), color='black');
    ax10.set_xlim([0.0,1.05]);
    ax10.set_ylim([0,500]);
    ax10.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig10.tight_layout();
    fig10.savefig("Distribution of k (p = "+probability+") (10000 steps).png");
    
    # Histogram of Beta
    fig11, ax11 = plt.subplots(figsize = [8,6]);
    ax11.set_title('Distribution of β', fontsize='x-large',fontweight='bold');
    ax11.set_xlabel('β', fontsize='x-large');
    ax11.set_ylabel('Frequency', fontsize='x-large');
    ax11.hist(Beta_history, bins=list(np.linspace(0,50,11)), color='black', alpha=0.7);
    ax11.set_xlim([0,52]);
    ax11.set_ylim([0,500]);
    ax11.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig11.tight_layout();
    fig11.savefig("Distribution of β (p = "+probability+") (10000 steps).png");        

    # Plot the histogram of the changes in population size
    fig14, ax14 = plt.subplots(figsize = [8,6]);
    ax14.set_title('Distribution of the change in the number of species in a time-step', fontsize='x-large',fontweight='bold');
    ax14.set_xlabel('Difference in the number of species in the system', fontsize='x-large');
    ax14.set_ylabel('Frequency', fontsize='x-large');
    num_n_diff = np.arange(1,len(n_diff)+1);
    ax14.set_yscale('log');
    ax14.bar(num_n_diff, n_diff, align='center', alpha=0.5, color='black');
    ax14.set_xticks(np.arange(0,20,2));
    ax14.set_xlim([0,18]);
    ax14.set_ylim([0,2*10**4]);
    ax14.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig14.savefig("Distribution of the change in the number of species in a time-step (p = "+probability+") (10000 steps).png");
    
    # Plot the histogram of extinctions
    fig15, ax15 = plt.subplots(figsize = [8,6]);
    ax15.set_title('Distribution of extinctions', fontsize='x-large',fontweight='bold');
    ax15.set_xlabel('Number of extinctions/time-step', fontsize='x-large');
    ax15.set_ylabel('Frequency', fontsize='x-large');
    num_Death_num = np.arange(1,len(Death_num)+1);
    ax15.set_yscale('log');
    ax15.bar(num_Death_num, Death_num, align='center', alpha=0.5, color='black');
    ax15.set_xticks(np.arange(0,20,2));
    ax15.set_xlim([0,18]);
    ax15.set_ylim([0,2*10**4]);
    ax15.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig15.savefig("Distribution of extinctions (p = "+probability+") (10000 steps).png");

#######################################################begin     
#     # Plot the evolution of population of bacteria that formed stable systems (fast vs slow)
#     fig16, ax16 = plt.subplots(figsize = [8,6]);

#     T = np.arange(1000)
#     bacteria_history[bacteria_history == 0] = np.nan
#     ax16.grid();
#     ax16.set_title('Evolution of population of bacteria (fast growers (red) vs slow growers (green))');
#     ax16.set_xlabel('time(generations of stable systems)');
#     ax16.set_ylabel('population');
#     ax16.set_yscale('log');
#     meso_b = round(num_bact/2)
#     for i in range(num_bact):
#         if i < meso_b:
#             ax16.plot(T, bacteria_history[i,:],'-ok', alpha=0.75, markersize=1.5, linewidth=0.65, color='green');
#         else:    
#             ax16.plot(T, bacteria_history[i,:],'-ok', alpha=0.75, markersize=1.5, linewidth=0.65, color='red');
            
#     fig16.savefig("Evolution of population of bacteria (fast vs slow) (p = 0.).png");  
    
#     # Plot the evolution of population of phages that formed stable systems (fast vs slow)
#     fig17, ax17 = plt.subplots(figsize = [8,6]);

#     phages_history[phages_history == 0] = np.nan
#     ax17.grid();
#     ax17.set_title('Evolution of population of phages (fast growers (red) vs slow growers (green))');
#     ax17.set_xlabel('time(generations of stable systems)');
#     ax17.set_ylabel('population');
#     ax17.set_yscale('log');
#     meso_p = round(num_phages/2)
#     for i in range(num_phages):
#         if i < meso_p:
#             ax17.plot(T, phages_history[i,:],'-ok', alpha=0.75, markersize=1.5, linewidth=0.65, color='green');
#         else:    
#             ax17.plot(T, phages_history[i,:],'-ok', alpha=0.75, markersize=1.5, linewidth=0.65, color='red');
        
#     fig17.savefig("Evolution of population of phages (fast vs slow) (p = 0.).png");  
###################################################################end

    # Plot the evolution of k
    fig18, ax18 = plt.subplots(figsize = [8,6]);

    k_t[k_t == 0] = np.nan
    ax18.grid();
    ax18.set_title('Evolution of k', fontsize='x-large',fontweight='bold');
    ax18.set_xlabel('time (generations of stable systems)', fontsize='x-large');
    ax18.set_ylabel('k', fontsize='x-large');
    for i in range(num_bact):
        ax18.plot(T, k_t[i,:],'-ok', markersize=1, linewidth=0.65);
        
    ax18.set_xlim([-500,10500]);
    ax18.set_ylim([0.0,1.2]);
    ax18.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig18.savefig("Evolution of k (p = "+probability+") (10000 steps).png");  
    
    # Plot the evolution of Beta
    fig19, ax19 = plt.subplots(figsize = [8,6]);

    Beta_t[Beta_t == 0] = np.nan
    ax19.grid();
    ax19.set_title('Evolution of β', fontsize='x-large',fontweight='bold');
    ax19.set_xlabel('time (generations of stable systems)', fontsize='x-large');
    ax19.set_ylabel('β', fontsize='x-large');
    for i in range(num_phages):
        ax19.plot(T, Beta_t[i,:],'-ok', markersize=1, linewidth=0.65);
        
    ax19.set_xlim([-500,10500]);
    ax19.set_ylim([0,60]);
    ax19.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig19.savefig("Evolution of Beta (p = "+probability+") (10000 steps).png"); 
    
    # Histogram of the percentage of extinct species at every timestep
    fig20, ax20 = plt.subplots(figsize = [8,6]);
    ax20.set_title('Distribution of the percentage of species that go extinct at every timestep', fontsize='large',fontweight='bold');
    ax20.set_xlabel('% of species that go extinct', fontsize='x-large');
    ax20.set_ylabel('Frequency', fontsize='x-large');
    ax20.hist(per_cent_extinct, bins=list(np.linspace(0,1,6)), color='black', alpha=0.7);
    ax20.set_xlim([0.0,1.05]);
    ax20.set_ylim([0,8400]);
    ax20.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig20.tight_layout();
    fig20.savefig("Distribution of % of extinctions (p = "+probability+") (10000 steps).png");   
    
    #histogram of k analysis
    hist_k1,edges_k1 = np.histogram(k_history,bins=list(np.linspace(0,1,11)))
    print(hist_k1)
    print(edges_k1)
    edges_k1 = np.delete(edges_k1,0)
    edges_k1 = np.delete(edges_k1,0)
    edges_k1 = np.delete(edges_k1,8)
    hist_k1 = np.delete(hist_k1,0)
    hist_k1 = np.delete(hist_k1,8)
    fig30, ax30 = plt.subplots(figsize = [8,6])
    ax30.scatter(edges_k1,hist_k1);
    ax30.set_ylim(bottom=0);
    slope_k1, intersect_k1 = np.polyfit(edges_k1,hist_k1, 1)
    Slope_k1 = slope_k1/hist_k1[0]
    ax30.plot(edges_k1, slope_k1*edges_k1+intersect_k1, color='red',label = 'slope ='+str(round(Slope_k1,3))+'\nintersect ='+str(round(intersect_k1,3)));
    print('slope of k_1 =',Slope_k1)
    print('intersect of k_1 =',intersect_k1)
    ax30.set_xlabel('k');
    ax30.set_ylabel('Frequency');
    ax30.legend();
    
    #histogram of Beta analysis
    hist_beta1,edges_beta1 = np.histogram(Beta_history,bins=list(np.linspace(0,50,11)))
    print(hist_beta1)
    print(edges_beta1)
    edges_beta1 = np.delete(edges_beta1,0)
    edges_beta1 = np.delete(edges_beta1,0)
    edges_beta1 = np.delete(edges_beta1,8)
    hist_beta1 = np.delete(hist_beta1,0)
    hist_beta1 = np.delete(hist_beta1,8)
    fig31, ax31 = plt.subplots(figsize = [8,6])
    ax31.scatter(edges_beta1,hist_beta1);
    ax31.set_ylim(bottom=0);
    slope_beta1, intersect_beta1 = np.polyfit(edges_beta1,hist_beta1, 1)
    Slope_beta1 = slope_beta1/hist_beta1[0]
    ax31.plot(edges_beta1, slope_beta1*edges_beta1+intersect_beta1, color='red',label = 'slope ='+str(round(Slope_beta1,3))+'\nintersect ='+str(round(intersect_beta1,3)));
    print('slope of β_1 =',Slope_beta1)
    print('intersect of β_1 =',intersect_beta1)
    ax31.set_xlabel('β');
    ax31.set_ylabel('Frequency');
    ax31.legend();
    
    # Histogram of k (every timestep counts)
    fig32, ax32 = plt.subplots(figsize = [8,6]);
    ax32.set_title('Distribution of k', fontsize='x-large',fontweight='bold');
    ax32.set_xlabel('k', fontsize='x-large');
    ax32.set_ylabel('Frequency', fontsize='x-large');
    ax32.hist(k_t[k_t!=0], bins=list(np.linspace(0,1,11)), color='black');
    ax32.set_xlim([0.0,1.05]);
    ax32.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig32.tight_layout();
    fig32.savefig("Histogram of k (every timestep counts) (p = "+probability+")%d.png"%epanal);
    np.savetxt("k hist p = %s repet = %d.txt" % (probability, epanal), k_t[k_t!=0], newline="\n")
    
    #histogram of k analysis 2
    hist_k1,edges_k1 = np.histogram(k_t[k_t!=0],bins=list(np.linspace(0,1,11)))
    print(hist_k1)
    print(edges_k1)
    file = open("hist_k p = %s.txt" % probability, "a")
    for i in hist_k1:
        file.write("%f, "%i)
    file.write('\n\n\n')
    file.close()
    file = open("edges_k p = %s.txt" % probability, "a")
    for i in edges_k1:
        file.write("%f, "%i)
    file.write('\n\n\n')    
    file.close()
    edges_k1 = np.delete(edges_k1,0)
    edges_k1 = np.delete(edges_k1,0)
    edges_k1 = np.delete(edges_k1,8)
    hist_k1 = np.delete(hist_k1,0)
    hist_k1 = np.delete(hist_k1,8)
    fig33, ax33 = plt.subplots(figsize = [8,6])
    ax33.scatter(edges_k1,hist_k1);
    ax33.set_ylim(bottom=0);
    slope_k1, intersect_k1 = np.polyfit(edges_k1,hist_k1, 1)
    Slope_k1 = slope_k1/hist_k1[0]
    ax33.plot(edges_k1, slope_k1*edges_k1+intersect_k1, color='red',label = 'slope ='+str(round(Slope_k1,3))+'\nintersect ='+str(round(intersect_k1,3)));
    print('slope of k_1 =',Slope_k1)
    print('intersect of k_1 =',intersect_k1)
    ax33.set_xlabel('k');
    ax33.set_ylabel('Frequency');
    ax33.legend();
    file = open("Slope_k1 p = %s.txt" % probability, "a") 
    file.write("%f, "%Slope_k1)
    file.close()
    
    # Histogram of k (log)
    fig34, ax34 = plt.subplots(figsize = [8,6]);
    ax34.set_title('Distribution of k', fontsize='x-large',fontweight='bold');
    ax34.set_xlabel('k', fontsize='x-large');
    ax34.set_ylabel('Frequency', fontsize='x-large');
    ax34.set_yscale('log')
    ax34.hist(k_t[k_t!=0], bins=list(np.linspace(0,1,11)), color='black');
    ax34.set_xlim([0.0,1.05]);
    #ax34.set_ylim([0,60]);
    ax34.legend(title='p = '+probability, frameon=False, title_fontsize = 'x-large', loc = 'best');
    plt.xticks(fontsize = 'large');
    plt.yticks(fontsize = 'large');
    fig34.tight_layout();
    fig34.savefig("logarithmic-Histogram of k (every timestep counts) model 2 (p = "+probability+")%d.png"%epanal);

In [ ]:
#def main():
print('Model 1 (10000 steps)')

# Set probability for a phage to attack a bacteria apart from the last one that entered the system
probability = "0.4" #don't forget to set the probability at Eta_matrix function !

print('Probability =',probability,'\n\n')

epanal = 11 # which of the repetitions for statistics

# Definition of timestep
Dt = 0.01

# Definition of rescaled parameters
Delta = 1  # Dilution constant for phage strains
alpha = 0.1  # Dilution constant for bacterial strains

#Initiation of all the containers of useful information about the species of the system
n = 0
species = np.array([], dtype=np.int64)
pop = np.array([], dtype=np.double)
k = np.array([], dtype=np.double)
dS_dt = np.array([], dtype=np.double)
Beta = np.array([], dtype=np.double)
eta_matrix = np.zeros([n,n], dtype=np.double)
t = 0
N = np.zeros(20, dtype=np.int64)          # The number of species
n_diff = np.zeros(20, dtype=np.int64)     # Changes in population size
Death_num = np.zeros(20, dtype=np.int64)  # Number of species that went extinct
per_cent_extinct = np.array([], dtype=np.double)  #percentage of species that went extict at every time step
eta_history = []
eta_invader = []
PB_interactions = [] #The number of bacteria that each phage infects for each stable system
Populo = np.zeros([20000,20002], dtype=np.double) # the first 10000 columns keep track of the population at every
                                                  # time step while the 10001,10002 columns keeps track of the characteristics.
                                                  # the last 10000 columns keep track of the number (age hierarchy) of species.

bacteria_history = np.array([], dtype=np.double)
phages_history = np.array([], dtype=np.double)
k_history = np.array([], dtype=np.double)
Beta_history = np.array([], dtype=np.double)
max_k = np.zeros(10000, dtype=np.double)
bbd1 = np.zeros([10000,2], dtype=np.int64) #places where Big Boss Dies (fastest growing bacteria) and is 
                                           #substituted by a faster growing bacteria
bbd2 = np.zeros([10000,2], dtype=np.int64) #places where Big Boss Dies (fastest growing bacteria) and is 
                                           #substituted by a slower growing bacteria

temp = np.empty_like(dS_dt)
result = np.zeros_like(dS_dt)

c = 0
c2 = 0
prev_n = 0
death_note = 0  #keeping track of the number of dead species at every timestep
n_max = 0
break_nar = 0
N_ALL = [0]*10000  #number of species (and what species) at every timestep
N_BACTERIA = [0]*10000
N_PHAGES = [0]*10000


COUNT2_or_more = 0 #number of phages
P_diff_B = 0 #cases where phages use different bacteria as their main food source
B_diff_P = 0 #cases where each bacterium is the main food source of a different phage 
both = 0 #cases where both of the above happen
same = 0 #cases where both happen and they are the same


while c < 10000:

    equillibrium = 2 # 2 means non-equillibrium, while 1 means equillibrium

    Beta, dS_dt, k, pop, result, species, temp, eta_matrix, n = add_new_species(Beta, dS_dt, k, pop, result, 
                                                                             species, temp, eta_matrix, n)

    check4eq(n, k, Beta) #check that we don't introduce a species to the system that it already exists

    #print('new species',species[n-1],'\n')

    Populo = keep_track(n, c, k, Beta, species, pop, Populo)
    
    eta_invader.append(eta_matrix)
    
    prev_n = n

    while equillibrium == 2:

        if n == 1:
            equillibrium = n1(pop, species, n, k, alpha, equillibrium)

        elif n > 1:
            A = np.zeros([n,n], dtype=np.double)
            K = np.zeros(n, dtype=np.double) 

            num_p, num_b = species_and_K(species, n, k, Beta, K)

            A = matrixA(species, n, A, K, eta_matrix, num_b)
            DetA = np.linalg.det(A)
            #print('Det',DetA)
            if DetA != 0:
                InvA = linalg.inv(A)
                S = np.zeros(n, dtype=np.double)
                equillibrium = Eq_Or_Dead(pop, species, K, S, n, InvA, alpha, Delta, equillibrium)



        if equillibrium == 2:
            rk4(pop, t, Dt, species, dS_dt, n, k, Beta, alpha, Delta, temp, result, eta_matrix)
            Beta, dS_dt, k, pop, result, species, temp, eta_matrix, Populo = resize_arrays(Beta, dS_dt, k, pop,
                                                                                           result,species, temp,
                                                                                           eta_matrix, n, 
                                                                                           Populo, c, c2)

            c2 += 1
            new_len_pop = len(pop)
            death_note += (n - new_len_pop)
            n = new_len_pop
            if n == 0:
                break;

        else:
            Populo = keep_track(n, c, k, Beta, species, pop, Populo)


    per_cent_extinct, death_note, n, Death_num, N, n_diff, n_max, break_nar = sp_dist(n, death_note, per_cent_extinct,
                                                                                      N, prev_n, n_diff, Death_num,
                                                                                      n_max, species, break_nar)
    
    #print(species,'\n')
    #print(k)
    #print(Beta)
    #print(pop,'\n')
    
    COUNT2_or_more, P_diff_B, B_diff_P, both, same = major_interactions(k, Beta, eta_matrix, COUNT2_or_more, 
                                                                        P_diff_B, B_diff_P, both, same)
    
    eta_history.append(eta_matrix)
    PB_interactions.extend((eta_matrix != 0).sum(1)) #sum non zero elements of rows (phages)
    
    N_ALL[c] = n
    N_BACTERIA[c] = np.count_nonzero(k)
    c += 1


# preparing the data for plotting
Populo = Populo[~np.all(Populo[:,:10000] == 0, axis=1)]
Populo[:,10000] = Populo[:,10000] - 1
Populo = Populo[Populo[:,10001].argsort()]
bacteria_history = Populo[Populo[:,10000] == 0]
phages_history = Populo[Populo[:,10000] == 1]
k_history = bacteria_history[:,10001]
Beta_history = phages_history[:,10001]
age_hierarchy = Populo[:,10002:] 
Populo = Populo[:,:10000]
bacteria_history = bacteria_history[:,:10000]
phages_history = phages_history[:,:10000]
k_t = np.zeros_like(bacteria_history)
Beta_t = np.zeros_like(phages_history)
num_Pop = len(Populo[:,0])
num_bact = len(bacteria_history[:,0])
num_phages = len(phages_history[:,0])
k_t, Beta_t = kappa_beta(k_t, Beta_t, bacteria_history, phages_history, k_history, Beta_history,
                         num_bact, num_phages)
calc_max_k(k_t, max_k, num_bact, bbd1, bbd2)
bbd1 = bbd1[~np.all(bbd1[:,:] == 0, axis=1)]
bbd2 = bbd2[~np.all(bbd2[:,:] == 0, axis=1)]
N_PHAGES = [x - y for x, y in zip(N_ALL, N_BACTERIA)]

print('Number of systems with more than 2 phages =',COUNT2_or_more)
print('Number of systems where each phage has a different main food source =',P_diff_B)
print('percentage =',P_diff_B/COUNT2_or_more,'\n')
print('Number of systems where each bacteria has a different main predator =',B_diff_P)
print('percentage =',B_diff_P/COUNT2_or_more,'\n')
print('Number of cases were both happen =',both)
print('percentage =',both/COUNT2_or_more,'\n')
difference = B_diff_P/COUNT2_or_more - both/COUNT2_or_more
print('difference % between phages and bacteria =',difference,'\n')
print('percentage both and the same =',same/COUNT2_or_more,'\n')
difference2 = both/COUNT2_or_more - same/COUNT2_or_more
print('difference % between both and both_and_the_same) =',difference2,'\n')
print('Total number of cases that follow this pattern =',P_diff_B/COUNT2_or_more + difference,'\n\n')

print('Average number of interactions per phage', sum(PB_interactions)/len(PB_interactions),'\n\n')

print('number of species that ever existed =', num_Pop)
print('number of bacteria that ever existed =', num_bact)
print('number of phages that ever existed =', num_phages,'\n\n')

print('Number of times that a fastest growing bacteria was substituted by an even faster growing one =', len(bbd1))
print('Number of times that a fastest growing bacteria was substituted by an slower growing one =', len(bbd2),'\n\n')

print('Average number of species =', sum(N_ALL)/len(N_ALL))
print('Average number of bacteria =', sum(N_BACTERIA)/len(N_BACTERIA))
print('Average number of phages =', sum(N_PHAGES)/len(N_PHAGES),'\n\n')

print('n_max =',n_max)


plots(species, pop, k, Beta, dS_dt, Populo, bacteria_history, phages_history, k_history, Beta_history, num_Pop,
      num_bact, num_phages, k_t, Beta_t, eta_history, max_k, bbd1, bbd2, per_cent_extinct, N, n_diff, Death_num,
      n_max, break_nar, probability)


In [ ]:
# #%%pixie_debugger
# #for i in range(20):
# if __name__ == '__main__':
#     t1 = time.time()
#     main()
#     print('Took', (time.time() - t1)/60)

In [ ]:
# calculating basic statistics for number of species
N_array = np.zeros(np.sum(N))
begin = 0
for i in range(len(N)):
    N_array[begin:begin+N[i]] = i
    begin += N[i]

print('mean number of species =', round(np.mean(N_array),3),' +- ',
      round(np.std(N_array)/np.sqrt(len(N_array)),3),'\nVariance =',
      round(np.var(N_array),3))    

In [ ]:
len(bbd1)

In [ ]:
bbd1

In [ ]:
#fastest growing bacteria gets substituted by an even fastest growing one
Random = randint(0, len(bbd1))
print(Random)
k_t[np.isnan(k_t)] = 0
Beta_t[np.isnan(Beta_t)] = 0
bacteria_history[np.isnan(bacteria_history)] = 0
phages_history[np.isnan(phages_history)] = 0
print('eta matrix after the substitution =\n', eta_history[bbd1[Random,0]])
print('eta matrix after the introduction of the potential invader =\n', eta_invader[bbd1[Random,0]])
print('eta matrix before the substitution =\n', eta_history[bbd1[Random,1]],'\n\n')
print('bacterial growth rates after the substitution = \n', k_t[np.nonzero(k_t[:,bbd1[Random,0]])[0],bbd1[Random,0]],'\n\n')
print('bacterial growth rates before the event = \n', k_t[np.nonzero(k_t[:,bbd1[Random,1]])[0],bbd1[Random,1]],'\n\n')
print('phages beta after the substitution = \n', Beta_t[np.nonzero(Beta_t[:,bbd1[Random,0]])[0],bbd1[Random,0]],'\n\n')
print('phages beta before the event = \n', Beta_t[np.nonzero(Beta_t[:,bbd1[Random,1]])[0],bbd1[Random,1]],'\n\n')
print('population of bacteria after the substitution = \n', bacteria_history[np.nonzero(bacteria_history[:,bbd1[Random,0]])[0],bbd1[Random,0]],'\n\n')
print('population of bacteria before the event = \n', bacteria_history[np.nonzero(bacteria_history[:,bbd1[Random,1]])[0],bbd1[Random,1]],'\n\n')
print('population of phages after the substitution = \n', phages_history[np.nonzero(phages_history[:,bbd1[Random,0]])[0],bbd1[Random,0]],'\n\n')
print('population of phages before the event = \n', phages_history[np.nonzero(phages_history[:,bbd1[Random,1]])[0],bbd1[Random,1]],'\n\n')

In [ ]:
len(bbd2)

In [ ]:
bbd2

In [ ]:
#fastest growing bacteria gets substituted by a slower growing one
Random = randint(0, len(bbd2))
print(Random)
k_t[np.isnan(k_t)] = 0
Beta_t[np.isnan(Beta_t)] = 0
bacteria_history[np.isnan(bacteria_history)] = 0
phages_history[np.isnan(phages_history)] = 0
print('eta matrix after the substitution =\n', eta_history[bbd2[Random,0]])
print('eta matrix after the introduction of the potential invader =\n', eta_invader[bbd2[Random,0]])
print('eta matrix before the substitution =\n', eta_history[bbd2[Random,1]],'\n\n')
print('bacterial growth rates after the substitution = \n', k_t[np.nonzero(k_t[:,bbd2[Random,0]])[0],bbd2[Random,0]],'\n\n')
print('bacterial growth rates before the event = \n', k_t[np.nonzero(k_t[:,bbd2[Random,1]])[0],bbd2[Random,1]],'\n\n')
print('phages beta after the substitution = \n', Beta_t[np.nonzero(Beta_t[:,bbd2[Random,0]])[0],bbd2[Random,0]],'\n\n')
print('phages beta before the event = \n', Beta_t[np.nonzero(Beta_t[:,bbd2[Random,1]])[0],bbd2[Random,1]],'\n\n')
print('population of bacteria after the substitution = \n', bacteria_history[np.nonzero(bacteria_history[:,bbd2[Random,0]])[0],bbd2[Random,0]],'\n\n')
print('population of bacteria before the event = \n', bacteria_history[np.nonzero(bacteria_history[:,bbd2[Random,1]])[0],bbd2[Random,1]],'\n\n')
print('population of phages after the substitution = \n', phages_history[np.nonzero(phages_history[:,bbd2[Random,0]])[0],bbd2[Random,0]],'\n\n')
print('population of phages before the event = \n', phages_history[np.nonzero(phages_history[:,bbd2[Random,1]])[0],bbd2[Random,1]],'\n\n')

In [ ]:
phages_history[np.nonzero(phages_history[:,bbd2[Random,1]])[0],bbd2[Random,1]]

In [ ]:
bacteria_history[np.nonzero(bacteria_history[:,bbd2[Random,1]])[0],bbd2[Random,1]]

In [ ]:
'''Statistics and plots that give insight for the cases where the fastest growing bacteria gets substituted
   by a slower growing one. What is the structure of those interaction networks.'''
   # check the dots that are outside the triangular region to see that they are indeed inside but has been a mistake in 
   # identifying the correct triangled that cannot be minimized to zero.

fig21, ax21 = plt.subplots(figsize=[8,6])
ax21.set_xlabel(r"$\dfrac{\eta_{R}}{\eta_{S}}$", fontsize=18)
ax21.set_ylabel(r"$\dfrac{k_{R}}{k_{S}}$", fontsize=18)
ax21.set_xlim([0, 1])
ax21.set_ylim([0, 1])
ax21.tick_params(axis='both', labelsize=18)

phage_intro = 0
bacteria_intro = 0
crazy_case = 0
crazy_case_2 = 0
classic_case = 0
two_bacteria_cases = 0
three_bacteria_cases = 0
FourOrMore_bacteria_cases = 0
eta_ratio = []
growth_rate_ratio = []
eta_ratio_2 = []
growth_rate_ratio_2 = []
eta_ratio_3 = []
growth_rate_ratio_3 = []
eta_ratio_4more = []
growth_rate_ratio_4more = []
total_cases = len(bbd2)


for i in range(total_cases):
    #check if it was a phage or bacterium that lead to the substitution of the fastest grower
    if eta_invader[bbd2[i,0]].shape[0] > eta_history[bbd2[i,1]].shape[0]:
        phage_intro += 1
        #visualize the effect of adding a phage (partial resistance in a virulent world-like plot)
        if k_t[np.nonzero(k_t[:,bbd2[i,0]])[0],bbd2[i,0]][0] != k_t[np.nonzero(k_t[:,bbd2[i,1]])[0],bbd2[i,1]][0]:
            crazy_case += 1
        elif eta_history[bbd2[i,1]].shape == (1,2) and eta_invader[bbd2[i,0]].shape == (2,2) and eta_history[bbd2[i,0]].shape == (1,1) and eta_invader[bbd2[i,0]][0][1] == 0:
            classic_case += 1
            eta_ratio.append(eta_invader[bbd2[i,0]][1][1] / eta_invader[bbd2[i,0]][1][0])
            growth_rate_ratio.append(k_t[np.nonzero(k_t[:,bbd2[i,1]])[0],bbd2[i,1]][0] / k_t[np.nonzero(k_t[:,bbd2[i,1]])[0],bbd2[i,1]][1])
        elif eta_history[bbd2[i,1]].shape[1] == 2:
            two_bacteria_cases += 1
            if eta_history[bbd2[i,0]][0][0] == eta_invader[bbd2[i,0]][-1][0]:
                eta_ratio_2.append(eta_history[bbd2[i,0]][0][0] / eta_invader[bbd2[i,0]][-1][1])
            else:
                eta_ratio_2.append(eta_history[bbd2[i,0]][0][0] / eta_invader[bbd2[i,0]][-1][0])
            growth_rate_ratio_2.append(k_t[np.nonzero(k_t[:,bbd2[i,1]])[0],bbd2[i,1]][0] / k_t[np.nonzero(k_t[:,bbd2[i,1]])[0],bbd2[i,1]][1])    
        elif np.count_nonzero(eta_history[bbd2[i,0]][-1]) == 1:
            if eta_history[bbd2[i,1]].shape[1] == 3:
                three_bacteria_cases += 1
                sort_k = sorted(k_t[np.nonzero(k_t[:,bbd2[i,1]])[0],bbd2[i,1]], reverse=True)
                for j in range(2):
                    if sort_k[j] not in k_t[np.nonzero(k_t[:,bbd2[i,0]])[0],bbd2[i,0]]:
                        if eta_invader[bbd2[i,0]][-1][j] != 0:
                            eta_ratio_3.append(eta_history[bbd2[i,0]][-1][-1] / eta_invader[bbd2[i,0]][-1][j])
                            growth_rate_ratio_3.append(sort_k[-1] / sort_k[j])
         
            elif eta_history[bbd2[i,1]].shape[1] >= 4:
                FourOrMore_bacteria_cases += 1
                sort_k = sorted(k_t[np.nonzero(k_t[:,bbd2[i,1]])[0],bbd2[i,1]], reverse=True)
                for j in range(len(sort_k)-1):
                    if sort_k[j] not in k_t[np.nonzero(k_t[:,bbd2[i,0]])[0],bbd2[i,0]]:
                        if eta_invader[bbd2[i,0]][-1][j] != 0:
                            eta_ratio_4more.append(eta_history[bbd2[i,0]][-1][-1] / eta_invader[bbd2[i,0]][-1][j])
                            growth_rate_ratio_4more.append(sort_k[-1] / sort_k[j]) 
                            
        
        else:
            crazy_case_2 += 1
                        
                        
    elif eta_invader[bbd2[i,0]].shape[1] > eta_history[bbd2[i,1]].shape[1]:
        bacteria_intro += 1
        print('BACTERIA')
        print('eta matrix after the substitution =\n', eta_history[bbd2[i,0]])
        print('eta matrix after the introduction of the potential invader =\n', eta_invader[bbd2[i,0]])
        print('eta matrix before the substitution =\n', eta_history[bbd2[i,1]],'\n\n')
        print('bacterial growth rates after the substitution = \n', k_t[np.nonzero(k_t[:,bbd2[i,0]])[0],bbd2[i,0]],'\n\n')
        print('bacterial growth rates before the event = \n', k_t[np.nonzero(k_t[:,bbd2[i,1]])[0],bbd2[i,1]],'\n\n')
        print('phages beta after the substitution = \n', Beta_t[np.nonzero(Beta_t[:,bbd2[i,0]])[0],bbd2[i,0]],'\n\n')
        print('phages beta before the event = \n', Beta_t[np.nonzero(Beta_t[:,bbd2[i,1]])[0],bbd2[i,1]],'\n\n')
        print('population of bacteria after the substitution = \n', bacteria_history[np.nonzero(bacteria_history[:,bbd2[i,0]])[0],bbd2[i,0]],'\n\n')
        print('population of bacteria before the event = \n', bacteria_history[np.nonzero(bacteria_history[:,bbd2[i,1]])[0],bbd2[i,1]],'\n\n')
        print('population of phages after the substitution = \n', phages_history[np.nonzero(phages_history[:,bbd2[i,0]])[0],bbd2[i,0]],'\n\n')
        print('population of phages before the event = \n', phages_history[np.nonzero(phages_history[:,bbd2[i,1]])[0],bbd2[i,1]],'\n\n')
    
    
print("percentage of cases where the substitution was initiated by a phage:",phage_intro/(phage_intro + bacteria_intro))
print("percentage of cases where the substitution was initiated by a bacterium:",bacteria_intro/(phage_intro + bacteria_intro))
print("total number of cases:", total_cases)
print("# phages:", phage_intro)
print("# bacteria:", bacteria_intro)
print('# classic cases: ', classic_case)
print('# cases with 2 bacteria: ', two_bacteria_cases)
print('# cases with 3 bacteria: ', three_bacteria_cases)
print('# cases with 4 or more bacteria: ', FourOrMore_bacteria_cases)
print('# cases where the last bacteria dies:', crazy_case)
print('# cases where the last phage attacks more than one bacteria in the final stable system:', crazy_case_2)                                                           
print('% of classic cases: ', classic_case/total_cases)
print('% of cases with 2 bacteria: ', two_bacteria_cases/total_cases)
print('% of cases with 3 bacteria: ', three_bacteria_cases/total_cases)
print('% of cases with 4 or more bacteria: ', FourOrMore_bacteria_cases/total_cases)
print('% of cases where the last bacteria dies:', crazy_case/total_cases)
print('% of cases where the last phage attacks more than one bacteria in the final stable system:', crazy_case_2/total_cases) 
print('% of situation where the fastest grower is substituted by a slower grower: ', total_cases/10000)

# ax21.plot(eta_ratio, growth_rate_ratio, 'r.', markersize=10, label = 'classic case');
# ax21.plot(eta_ratio_2, growth_rate_ratio_2, 'b.', markersize=10, label = 'cases with 2 bacteria');
# ax21.plot(eta_ratio_3, growth_rate_ratio_3, 'g.', markersize=10, label = 'cases with 3 bacteria');
# ax21.plot(eta_ratio_4more, growth_rate_ratio_4more, 'k.', markersize=10, label = 'cases with 4 bacteria or more');                                                           


ax21.plot(eta_ratio, growth_rate_ratio, 'r.', markersize=10);
ax21.plot(eta_ratio_2, growth_rate_ratio_2, 'r.', markersize=10);
ax21.plot(eta_ratio_3, growth_rate_ratio_3, 'r.', markersize=10);
ax21.plot(eta_ratio_4more, growth_rate_ratio_4more, 'r.', markersize=10);
ax21.legend(title='p = '+probability, fontsize = 20, title_fontsize = 20, loc='lower right', frameon=False);

fig21.savefig("ratio of etas VS ratio of growth rates (p = "+probability+")(10000 steps).png");